In [ ]:
!pip install folium geopandas scikit-learn pandas numpy shapely

In [2]:
import pandas as pd
import numpy as np
import json
import folium
from folium.plugins import HeatMap
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
DATA_FILES = [
    "PRSA_Data_Aotizhongxin_20130301-20170228.csv",
    "PRSA_Data_Changping_20130301-20170228.csv",
    "PRSA_Data_Dingling_20130301-20170228.csv",
    "PRSA_Data_Dongsi_20130301-20170228.csv",
    "PRSA_Data_Tiantan_20130301-20170228.csv"
]

DATA_DIR = "/content/data"
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Beijing station coordinates
STATION_COORDS = {
    "Aotizhongxin": (39.982, 116.397),
    "Changping": (40.218, 116.231),
    "Dingling": (40.299, 116.239),
    "Dongsi": (39.929, 116.417),
    "Tiantan": (39.886, 116.407)
}

In [4]:
all_data = []

for file in DATA_FILES:
    path = os.path.join(DATA_DIR, file)
    df = pd.read_csv(path)

    # Extract station name from file name
    station_name = file.split("_")[2] if "Data" in file else file.split("_")[1]
    lat, lon = STATION_COORDS[station_name]

    # Attach coordinates
    df["station"] = station_name
    df["lat"] = lat
    df["lon"] = lon

    all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)
print("Total rows loaded:", len(df_all))

Total rows loaded: 175320


In [6]:
df_all

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station,lat,lon
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,39.982,116.397
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,39.982,116.397
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,39.982,116.397
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,39.982,116.397
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,39.982,116.397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175315,35060,2017,2,28,19,20.0,48.0,2.0,NaN,500.0,NaN,12.5,1013.5,-16.2,0.0,NW,2.4,Tiantan,39.886,116.407
175316,35061,2017,2,28,20,11.0,34.0,3.0,36.0,500.0,NaN,11.6,1013.6,-15.1,0.0,WNW,0.9,Tiantan,39.886,116.407
175317,35062,2017,2,28,21,18.0,32.0,4.0,48.0,500.0,48.0,10.8,1014.2,-13.3,0.0,NW,1.1,Tiantan,39.886,116.407
175318,35063,2017,2,28,22,15.0,42.0,5.0,52.0,600.0,44.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Tiantan,39.886,116.407


In [8]:
df_pm25 = df_all[["station", "lat", "lon", "PM2.5"]].dropna()
df_pm25 = df_pm25[df_pm25["PM2.5"] >= 0]   # remove bad values

print("Remaining valid PM2.5 rows:", len(df_pm25))

Remaining valid PM2.5 rows: 171415


In [10]:
agg = df_pm25.groupby(["station", "lat", "lon"]).agg(
    mean_pm25=("PM2.5", "mean"),
    count=("PM2.5", "size")
).reset_index()

print("\nAggregated Sensor Table:")
print(agg)

agg.to_csv(os.path.join(OUTPUT_DIR, "sensor_aggregates.csv"), index=False)



Aggregated Sensor Table:
        station     lat      lon  mean_pm25  count
0  Aotizhongxin  39.982  116.397  82.773611  34139
1     Changping  40.218  116.231  71.099743  34290
2      Dingling  40.299  116.239  65.989497  34285
3        Dongsi  39.929  116.417  86.194297  34314
4       Tiantan  39.886  116.407  82.164911  34387


In [11]:
X = agg[["lat", "lon"]].values
y = agg["mean_pm25"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = KNeighborsRegressor(n_neighbors=3, weights="distance")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print("\nModel MAE:", mae)


Model MAE: 1.1572249228206317


In [12]:
lat_min, lat_max = agg['lat'].min(), agg['lat'].max()
lon_min, lon_max = agg['lon'].min(), agg['lon'].max()

grid_lats = np.linspace(lat_min - 0.03, lat_max + 0.03, 60)
grid_lons = np.linspace(lon_min - 0.03, lon_max + 0.03, 60)

grid_points = np.array([[lat, lon] for lat in grid_lats for lon in grid_lons])
grid_preds = model.predict(grid_points)

# Save grid predictions
grid_df = pd.DataFrame(grid_points, columns=["lat", "lon"])
grid_df["pm25_pred"] = grid_preds
grid_df.to_csv(os.path.join(OUTPUT_DIR, "pm25_grid_predictions.csv"), index=False)

In [13]:
features = []
for (lat, lon), val in zip(grid_points, grid_preds):
    features.append({
        "type": "Feature",
        "properties": {"pm25_pred": float(val)},
        "geometry": {"type": "Point", "coordinates": [float(lon), float(lat)]}
    })

geojson = {"type": "FeatureCollection", "features": features}
with open(os.path.join(OUTPUT_DIR, "pm25_predictions.geojson"), "w") as f:
    json.dump(geojson, f)

In [14]:
center_lat = agg['lat'].mean()
center_lon = agg['lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Heatmap from predictions
heat_data = [[lat, lon, float(pm)] for (lat, lon), pm in zip(grid_points, grid_preds)]
HeatMap(heat_data, radius=12, blur=18).add_to(m)

# Station markers
for _, row in agg.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"{row['station']} — mean PM2.5: {row['mean_pm25']:.1f}",
        icon=folium.Icon(color="blue")
    ).add_to(m)

map_path = os.path.join(OUTPUT_DIR, "beijing_pm25_interpolation_map.html")
m.save(map_path)

In [15]:
print("\nMap saved to:", map_path)


Map saved to: outputs/beijing_pm25_interpolation_map.html
